# Codename Hints

This notebook explores the strategies `EmbeddingSimilarity`, `MaskFilling` and `MaskFillingWithExclusion` and gives an intro to the general usage.

Note that implementations in the notebook and the codenames module may diverge.

## Game Words

We start with 25 random words from a list of ambiguous words.

In [ ]:
import random

random.seed(123)

from codenames.model import Game
from codenames.utils import print_game

game = Game.create_random()
print_game(game)

## Hint words

For possible hints, we want to search a big vocabulary.
The tokenizer of BERT (a WordPiece tokenizer) contains a lot of English words as well as inflections of words.

In [ ]:
from transformers import BertTokenizer, set_seed

set_seed(123)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
vocab = tokenizer.get_vocab()


# filter out non-word / non-English tokens and tokens contained in the game
def wordlike(s: str) -> bool:
    return s.isalpha() and len(s) > 2


hint_words = {
    t: idx for t, idx in vocab.items() if wordlike(t) and t not in game.all_words
}

print(f"Using {len(hint_words)} hint words")
print("Examples:", random.sample(sorted(hint_words), 20))
print("Game words missing in vocab:", [w for w in game.all_words if w not in vocab])

## Strategies

I do not have a dataset of useful Codenames hints, so I'll use pretrained models or unsupervised methods.

Given a game like the one above, a strategy should provide hints, together with a score and the intended target words.

```python
# model.py

class Hint(BaseModel):
    word: str           # the hint word
    score: float        # the quality of the hint
    targets: list[str]  # the game words targeted by the hint


# strategy.py

class HintStrategy:

    @abc.abstractmethod
    def __call__(self, game: Game, hint_for_red=True) -> Iterator[Hint]:
        pass

    ...
```

For the evaluation, we look at the top k results.

## Strategy 1: Similarities in the embedding space

One simple approach is just to look at pretrained **embeddings** and their similarities.

Targets are selected by their similarity to the hint word.
Then, the selection is scored, based on the categories of the selected words.
The Baseline strategy is configurable with weights for each category.

`cosine_similarity` is a common similarity measure that looks at the angles in the embedding vector space. Other similarities could be euclidean distance or dot product.

In [ ]:
from codenames.strategy import HintStrategy
from codenames.model import Hint, TeamWeights

from typing import Iterator, Callable

from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn

Words2Ids = Callable[[list[str]], list[int]]


class EmbeddingSimilarity(HintStrategy):
    default_weights = TeamWeights(team=1, other_team=-5, neutral=-2, assassin=-10)

    def __init__(
        self,
        embeddings: nn.Embedding,
        words2ids: Words2Ids,
        hint_words: dict[str, int],
        weights: TeamWeights | None = None,
    ):
        """

        :param embeddings: some pretrained embeddings
        :param words2ids: mapping game words to their embedding indices
        :param hint_words: a mapping of words and indices for the hints
        :param weights: factors for the incremental score calculation
        """
        self.embeddings = embeddings
        self.words2ids = words2ids
        self.hint_words = hint_words
        self.weights = weights or self.default_weights

    def __call__(self, game: Game, hint_for_red=True) -> Iterator[Hint]:
        words = game.all_words
        similarities = self.calculate_similarities(game)

        # incrementally calculate the score
        for hint_word, per_hint_similarities in zip(self.hint_words, similarities):
            score = 0
            targets = []
            # order by high similarity
            for similarity, word, cat in sorted(
                zip(per_hint_similarities, words, game.teams(hint_for_red)),
                reverse=True,
            ):
                targets.append(word)
                # The score is affected by the similarity itself
                # and the weight per category (which also affects the polarity)
                score += similarity * getattr(self.weights, cat)
                yield Hint(word=hint_word, score=score, targets=targets)

    def calculate_similarities(self, game):
        # calculate all similarities between hint words and game words
        with torch.no_grad():
            game_word_ids = self.words2ids(game.all_words)
            game_embs = self.embeddings(
                torch.tensor(game_word_ids)
            )  # shape: (25, n_emb)
            hint_word_embs = self.embeddings(
                torch.tensor(list(self.hint_words.values()))
            )  # shape: (n_vocab, n_emb)
            similarities = cosine_similarity(
                hint_word_embs, game_embs
            )  # shape (n_vocab, n_emb)
        return similarities

### Results

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased")
embeddings = model.get_input_embeddings()


def words2ids(words):
    return [s[1] for s in tokenizer(words)["input_ids"]]


embeddings_strategy = EmbeddingSimilarity(embeddings, words2ids, hint_words)

In [ ]:
embeddings_strategy.print_top_k(game, 10)

In [ ]:
embeddings_strategy.print_top_k(game, 10, hint_for_red=False)

### Evaluation

This already works much better than I had expected. In particular, I expected the polysemous nature of the game words to make this a very difficult task.

All top results only target words of their own team. Changing the weights might allow neutral words to be included but therefore target more team words as well.

## Strategy 2: MASK Filling

Another approach is to use the mask filling head for BERT.

For that, a subset of the game words is provided as a textual input and the model must predict a good fit.

A big problem is the number of possible subsets (25^2 = 33 Mio) - however, we can exclude subsets that:
* contain the assassin
* more than 2 words of the wrong color
* less than two words
  
An important choice is the template of the query; some possibilities:
* `"The words <words> are all closely related to [MASK], but not necessarily in the same way"`
* `"<words> are all related by [MASK]"`
* `"<words> and [MASK]"`

It is a general problem, that this strategy inverts the task.
It does not find game words given a hint, but a hint given a subset of game words.
A hint may be a very good fit for a subset but also for words not included in the subset - so it may hit the assassin more likely.

In [ ]:
from itertools import combinations, product
from transformers import pipeline
from typing import Iterator, Sequence
from scipy.special import comb

device = "cuda" if torch.cuda.is_available() else "cpu"

TEMPLATE = "The words {} can be thought of together with the word [MASK], maybe also through different semantic aspects."
SubsetGenerator = Iterator[list[str]]


class MaskFilling(HintStrategy):
    default_weights = TeamWeights(team=1, other_team=-3, neutral=-1, assassin=-100)

    def __init__(
        self,
        checkpoint="bert-base-uncased",
        weights: TeamWeights | None = None,
        max_good_targets: int = 4,
        max_bad_targets: int = 1,
        template=TEMPLATE,
    ):
        self.unmasker = pipeline("fill-mask", model=checkpoint, device=device)
        self.weights = weights or self.default_weights
        self.max_good_targets = max_good_targets
        self.max_bad_targets = max_bad_targets
        self.template = template

    def __call__(self, game: Game, hint_for_red=True) -> Iterator[Hint]:
        all_words = set(game.all_words)
        teams = game.teams(red=hint_for_red)
        word2score = {
            word: getattr(self.weights, cat) for word, cat in zip(game.all_words, teams)
        }

        team, other_team = (
            (game.red, game.blue) if hint_for_red else (game.blue, game.red)
        )
        subset_generator = lambda: self.candidate_subsets(
            team, other_team, game.neutral
        )
        with torch.no_grad():
            inputs = self.create_inputs(subset_generator, game, hint_for_red)
            all_results = self.unmasker(inputs)
            for results, subset in zip(all_results, subset_generator()):
                subset_score = sum(word2score[word] for word in subset)
                for result in results:
                    # exclude tokens are contained in the game
                    if result["token_str"] in all_words:
                        continue
                    yield Hint(
                        word=result["token_str"],
                        score=result["score"] * subset_score,
                        targets=subset,
                    )

    @staticmethod
    def words2string(words: Sequence[str]):
        return ", ".join(words[:-1]) + f" and {words[-1]}"

    def create_inputs(
        self,
        subset_generator: Callable[[], SubsetGenerator],
        game: Game,
        hint_for_red: bool,
    ) -> list[str]:
        return [
            self.template.format(self.words2string(subset))
            for subset in subset_generator()
        ]

    def candidate_subsets(
        self, team: set[str], other_team: set[str], neutral: set[str]
    ) -> SubsetGenerator:
        # subsets containing words of team
        def all_good_targets():
            for n_good_targets in range(2, min(self.max_good_targets, len(team)) + 1):
                for combination in combinations(team, n_good_targets):
                    yield combination

        # subsets containing words of other team or neutral
        def all_bad_targets():
            for n_bad_targets in range(self.max_bad_targets):
                for combination in combinations(
                    other_team.union(neutral), n_bad_targets + 1
                ):
                    yield combination

        for good_targets in all_good_targets():
            yield good_targets
            for bad_targets in all_bad_targets():
                yield good_targets + bad_targets

    def n_subsets(self, team: set[str], other_team: set[str], neutral: set[str]) -> int:
        n_good = len(team)
        n_bad = len(other_team.union(neutral))
        good_combos = sum(
            comb(n_good, k, exact=True)
            for k in range(2, min(self.max_good_targets, n_good) + 1)
        )
        bad_combos = sum(
            comb(n_bad, k, exact=True)
            for k in range(1, min(self.max_bad_targets, n_bad) + 1)
        )
        return good_combos + good_combos * bad_combos

In [ ]:
masking_strategy = MaskFilling()

### Results

In [ ]:
masking_strategy.print_top_k(game, 10)

In [ ]:
masking_strategy.print_top_k(game, 10, hint_for_red=False)

### Evaluation

These results actually show a deeper semantic understanding that Strategy 1. I had to experiment a bit with the template, to prevent grammatical terms or inflected forms of game words in the results, but I am quite content with the results.

Unfortunately, the results vary a lot with the nondeterministic components, so you may get completely different hints.
I liked the red hints:
* `earth` targeting `date`, `light` and `wave`
* `energy` targeting `bond`, `light` and `wave`
* `head` targeting `cap`, `part`, `wave` and `trunk` (though trunk is neutral)

and for blue:
* `witness` targeting `trial`, `bill`, `stand` and `chest`
* `truck` targeting `bill`, `chest`, `jam` and `plane`
* `defense` targeting `trial`, `stand` and `chest`
  
As mentioned above, this strategy will give hints that also target other terms. For example, `energy` can also be associated with `plane` of the other team, in some sense.
  
In a refined version, this can be excluded by a more elaborate input that also takes into account the other words.

## Strategy 3: MaskFilling with exclusion

In [ ]:
# We only have to adapt the template and include the 'bad words' (assassin or other team)
# The remaining behaviour is inherited from MaskFilling

EXCLUSION_TEMPLATE = (
    "The word [MASK] can be thought of together with the words {included}, "
    + "but does not relate to any of the words {excluded}."
)


class MaskFillingWithExclusion(MaskFilling):

    def create_inputs(
        self,
        subset_generator: Callable[[], SubsetGenerator],
        game: Game,
        hint_for_red: bool,
    ) -> list[str]:
        inputs = []
        bad_terms = set(game.blue if hint_for_red else game.red).union([game.assassin])
        for included in subset_generator():
            excluded = list(bad_terms.difference(included))
            inputs.append(
                self.template.format(
                    included=self.words2string(included),
                    excluded=self.words2string(excluded),
                )
            )
        return inputs

In [ ]:
masking_with_exclusion_strategy = MaskFillingWithExclusion(template=EXCLUSION_TEMPLATE)

In [ ]:
masking_with_exclusion_strategy.print_top_k(game, 25)

In [ ]:
masking_with_exclusion_strategy.print_top_k(game, 25, hint_for_red=False)

## Notes

* The MaskFilling Strategies can be configured:
    * `max_good_targets`: how many targets of own team at max?
    * `max_bad_targets`: how many neutral or words of the other team are allowed at max?
    * `template`: change the masking template